<a href="https://colab.research.google.com/github/DenCoder618/Humans-Now-Invited/blob/main/Humans_Now_Invited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Humans Not Invited (but Now invited!)

Сайт: http://humansnotinvited.com

In [ ]:
#@title Подготавливаем всё что надо
import requests as r
import json as j
from bs4 import BeautifulSoup
from hashlib import md5

!git clone https://github.com/DenCoder618/Humans-Now-Invited -q
with open("/content/Humans-Now-Invited/tags.json") as f:
  tags = j.loads(f.read())

url = "http://www.humansnotinvited.com/"
concat_list = lambda a: [j for i in a for j in i] # полезная функция, преобразует [[..], [.], [...]] в [......]

arr = {k:sorted( # вот тут мы как раз получаем данные о статистическом распределении
  [[i, concat_list(v).count(i)] for i in list(set(concat_list(v)))], key=lambda e: e[1]
)[::-1][:9] for k, v in tags.items()}

tictactoefy = lambda a: " _______\n| " + \
  "|\n| ".join(["".join(["x " if i+1 in a else "  " for i in range(9)][i:i+3]) for i in (0,3,6)])+"|\n ‾‾‾‾‾‾‾"

# def tictactoefy(a):
#   t = ["x " if i in a else "  " for i in range(1, 10)]
#   return " _______\n| " + "".join(t[:3]) + "|\n| " + "".join(t[3:6]) + "|\n| " + "".join(t[6:9]) + "|\n ‾‾‾‾‾‾‾"

In [ ]:
#@title [ Успешно ] проходим капчу
#@markdown Просто скопируйте исходный код страницы капчи (Ctrl + U; A; C) и вставьте в поле ниже
html = "" #@param {type:"string"}
bs = BeautifulSoup(html)
tag = bs.find("strong").text
print("Tag:", tag)
bs = [md5(r.get(url + i["src"]).content).hexdigest() for i in bs.find_all("img")[1:-1]]

a = set()
for i in arr[tag]:
  if i[0] in bs:
    a.add(bs.index(i[0])+1)
ans = sorted(list(a))
print("Answer:", " ".join([str(i) for i in ans]))
print(tictactoefy(ans))

# Как это устроено

Алгоритм работает на законах статистического распределения, то есть для обучения ему нужно много статистических данных о вариантах расположения картинок в капче (около 1000 примеров), после этого на основании данных о количестве появлений конкретной картинки с определенным тегом алгоритм делает вывод, к какому тегу эта картинка вероятно принадлежит ('точно' принадлежит, если нет пересечений, если же они есть, то есть одну картинку можно статистически отнести к двум разным тегам, выбирается тег с наибольшей вероятностью)

Конечно же можно было сделать более совершенный алгоритм, основанный не на статистике, но как гласит первое правило программистов: "Работает - не трогай!", да и эту версию я написал на 30 минут, пока остывал мой чай

Информация о капче: 24 темы, 216 картинок, 9 картинок на тему

```
Темы:
man, women, modems, storefronts, flowers, spinners, memes, trafficlights,
dogs, dicks (члены хаха), hotdogs, selfiesticks, kids, ewoks, cats, computers,
cars, art, ants, star trek, spiders, speakers, servers, emojis
```

# Дополнительно - тренируем

In [ ]:
cmax, c = 42, 0 # это не отсылка к Автостопу, точно-точно
# 24 * 42 = 1008 > 1000 - достаточно
while c < cmax:
  data = BeautifulSoup(r.get(url).content)
  # получить картинки
  urls, hashes = [i["src"] for i in data.find_all("img")][1:-1], []
  for u in urls:
    hashes.append(md5(r.get(url + u).content).hexdigest()) # получаем хэши

  tag = data.find("strong").text
  if tag not in tags.keys():
    tags.update({tag: [hashes]})
  else:
    tags[tag].append(hashes)
  
  c = len(concat_list(tags.values())) / len(tags.keys())
  print(len(concat_list(tags.values())), len(tags.keys()), round(c, 3), sep=" - ")

In [ ]:
with open("tags.json", "w") as f:
  f.write(j.dumps(tags)) # сохранение

In [ ]:
tags = {} # для очистки данных